# Analyzing Car Reviews with LLMs - JAZIL Notebook

![alt text](PictureCarReview.jpg)

### Car-ing is sharing, an auto dealership company for car sales and rental, is taking their services to the next level thanks to Large Language Models (LLMs). As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company. The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.

## The CTO at "Car-ing is sharing", a car sales and rental company requested to help prototype a chatbot app that addresses diverse inquiries using LLMs. 


#### Installing transformers and evaluate package from HuggingFace which provide range of ready to use LLMs for solving our problem and evaluating on our solution.

In [1]:
#!pip install transformers
#!pip install evaluate
#!pip install -v torch

In [5]:
from transformers import logging
logging.set_verbosity(logging.WARNING)
import pandas as pd
import torch
import os

C:\Users\jazil.kalim\my_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### LOADING DATA

In [6]:
##check directory
os.getcwd()
print(f'Current working directory is : {os.getcwd()}')

Current working directory is : C:\Users\jazil.kalim


In [7]:
file_path= 'D:\\Learning Courses\\DSProjects\\Datacamp Analyzing Car Reviews with LLMs\\data\\car_reviews.csv'
df=pd.read_csv(file_path, delimiter=';')
print(df.head())

                                              Review     Class
0  I am very satisfied with my 2014 Nissan NV SL....  POSITIVE
1  The car is fine. It's a bit loud and not very ...  NEGATIVE
2  My first foreign car. Love it, I would buy ano...  POSITIVE
3  I've come across numerous reviews praising the...  NEGATIVE
4  I've been dreaming of owning an SUV for quite ...  POSITIVE


In [8]:
## PUTTING CAR REVIEWS AND THEIRE ASSOCIATED SENTIMENT LABELS IN TO LISTS
reviews=df['Review'].tolist()
real_labels=df['Class'].tolist()

### Section- I : SENTIMENT ANALYSIS
#### - Use a pre-trained LLM to classify the sentiment of the five car reviews in the car_reviews.csv dataset, and evaluate the classification accuracy and F1 score of predictions.
#### - Store the model outputs in predicted_labels, then extract the labels and map them onto a list of {0,1} integer binary labels called predictions.

In [9]:
#LOADING SENTIMENT ANALYSIS LLM into a pipeline
from transformers import pipeline
classifier=pipeline('sentiment-analysis',model='distilbert-base-uncased-finetuned-sst-2-english')

In [10]:
predicted_labels=classifier(reviews)

In [11]:
print(predicted_labels)

[{'label': 'POSITIVE', 'score': 0.929398238658905}, {'label': 'POSITIVE', 'score': 0.8654283285140991}, {'label': 'POSITIVE', 'score': 0.9994640946388245}, {'label': 'NEGATIVE', 'score': 0.9935314059257507}, {'label': 'POSITIVE', 'score': 0.9986565113067627}]


In [12]:
## PERFORMING INFERENCE ON CAR REVIEWS

for review, prediction, label in zip(reviews,predicted_labels,real_labels):
    print(f"{review}\nActual Sentiment: {label}\nPredicted Sentiment: {prediction['label']} (Confidence: {prediction['score']:.4f})\n-----------------------------\n")

I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel trailer li

In [13]:
### ACCURACY AND F1 SCORE METRICS
import evaluate


In [14]:
accuracy=evaluate.load('accuracy')
f1= evaluate.load('f1')

In [15]:
# Mapping categorical sentiment labels into integer labels
references= [1 if label=='POSITIVE' else 0 for label in real_labels]
predictions= [1 if label=='POSITIVE' else 0 for label in predicted_labels]

In [16]:
# Calculating accuracy and F1 score
accuracy_result=accuracy.compute(references=references,predictions=predictions)
f1_result=f1.compute(references=references,predictions=predictions)

In [17]:
print(f"Accuracy: {accuracy_result['accuracy']}")
print(f"F1: {f1_result['f1']}")

Accuracy: 0.4
F1: 0.0


### SECTION- II : TRANSLATION
#### The company is recently attracting customers from Spain. Extract and pass the first two sentences of the first review in the dataset to an English-to-Spanish translation LLM. Calculate the BLEU score to assess translation quality, using the content in reference_translations.txt as references.
#### - Store the translated text generated by the LLM in translated_review.
#### - Store the BLEU score metric result in bleu_score.

In [26]:
## LOADING TRANSLATION LLM AND TRANSLATING CAR REVIEW
first_review= reviews[0]
translator=pipeline('translation',model='Helsinki-NLP/opus-mt-en-es')

In [28]:
translated_review=translator(first_review,max_length=27)[0]['translation_text']

Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [30]:
print(f"Model Translation:\n {translated_review}")

Model Translation:
 Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


In [33]:
## LOADING REFERENCE TRANSLATION FROM FILE
with open('D:\\Learning Courses\\DSProjects\\Datacamp Analyzing Car Reviews with LLMs\\data\\reference_translations.txt','r') as file:
    lines= file.readlines()

In [36]:
references= [line.strip() for line in lines]

In [37]:
print(f"Spanish translation references: \n {references}")

Spanish translation references: 
 ['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']


In [39]:
### BLEU SCORE METRIC
bleu=evaluate.load('bleu')

In [43]:
bleu_score=bleu.compute(predictions=[translated_review],references=[references])
print(f"BLEU SCORE : {bleu_score['bleu']:.4f} ")

BLEU SCORE : 0.6023 


### Section- III: Extractive QA
#### The 2nd review in the dataset emphasizes brand aspects. Load an extractive QA LLM such as "deepset/minilm-uncased-squad2" to formulate the question "What did he like about the brand?" and obtain an answer.
#### Use question and context for the two variables containing the LLM inputs: question and context.
#### Store the actual text answer in answer.

In [45]:
# IMPORT AUTO CLASSES (CAN BE SOLVED VIA PIPELINE TOO)
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering

In [46]:
## INSTANTIATE MODEL AND TOKENIZER
model_ckp="deepset/minilm-uncased-squad2"
tokenizer=AutoTokenizer.from_pretrained(model_ckp)

C:\Users\jazil.kalim\my_venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jazil.kalim\.cache\huggingface\hub\models--deepset--minilm-uncased-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [47]:
model=AutoModelForQuestionAnswering.from_pretrained(model_ckp)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
## DEFINE QUESTION AND CONTEXT AND TOKENIZE THEM
context=reviews[1] ## review 2
print(f"Context:\n {context}")
      

Context:
 The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.


In [52]:
question = "What did he like about the brand?"
inputs= tokenizer(question,context,return_tensors='pt')

In [57]:
## PERFORMING INFERENCE AND EXTRACTING ANSWERS FROM RAW OUTPUTS
with torch.no_grad():
    outputs=model(**inputs)


In [64]:
start_idx=torch.argmax(outputs.start_logits)

In [69]:
end_idx= torch.argmax(outputs.end_logits)+1

In [73]:
answer_span=inputs['input_ids'][0][start_idx:end_idx]

In [78]:
##Decode the answer
answer = tokenizer.decode(answer_span)

In [80]:
print(f"Answer: \n {answer}")

Answer: 
 ride quality, reliability


### Section-IV : Text Summarization
#### Summarize the last review in the dataset, into approximately 50-55 tokens long. Store it in the variable summarized_text.

In [83]:
## GET THE ORIGNINAL TEXT TO SUMMARIZE
text_to_summarize= reviews[-1]
print(f"Original text to summarize: \n {text_to_summarize}")

Original text to summarize: 
 I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the 

In [85]:
##LOADING SUMMARIZATION PIPELINE and PERFORMING INFERENCE
model_name= "cnicu/t5-small-booksum"

In [87]:
summarizer= pipeline("summarization", model=model_name)

In [88]:
outputs=summarizer(text_to_summarize,max_length=53)


In [92]:
summarized_text= outputs[0]['summary_text']
print(f"Summarized text : \n {summarized_text}")

Summarized text : 
 the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more.
